# Dividend Kings Analysis

This notebook pulls dividend history using Yahoo Finance (via yfinance) and estimates when each company became a Dividend King (50+ years of increases).

In [ ]:
import yfinance as yf
import pandas as pd
import datetime

In [ ]:
dividend_kings = {
    "KO": "Coca-Cola",
    "JNJ": "Johnson & Johnson",
    "PG": "Procter & Gamble",
    "EMR": "Emerson Electric",
    "MMM": "3M",
    "GPC": "Genuine Parts",
    "LOW": "Lowe's",
    "CL": "Colgate-Palmolive"
}

In [ ]:
def get_dividend_king_info(ticker):
    stock = yf.Ticker(ticker)
    dividends = stock.dividends
    if dividends.empty:
        return {'Ticker': ticker, 'Name': dividend_kings[ticker], 'Years Increased': 0, 'Joined': None}
    dividends_by_year = dividends.resample('Y').sum()
    streak = 0
    previous = 0
    values = dividends_by_year.values[::-1]
    for i in range(len(values)):
        if values[i] > 0 and (previous == 0 or values[i] >= previous):
            streak += 1
            previous = values[i]
        else:
            break
    join_year = datetime.datetime.now().year - (streak - 50) if streak >= 50 else None
    return {'Ticker': ticker, 'Name': dividend_kings[ticker], 'Years Increased': streak, 'Joined': join_year}

In [ ]:
results = []
for ticker in dividend_kings:
    try:
        info = get_dividend_king_info(ticker)
        results.append(info)
    except Exception as e:
        results.append({'Ticker': ticker, 'Name': dividend_kings[ticker], 'Error': str(e)})
pd.DataFrame(results)